In [1]:
#Python packages
import numpy as np
import pandas as pd
import re
from scipy import stats
import json

#Visualization packages
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#modeling packages
from collections import defaultdict
import lightgbm as lgb
import copy
import xgboost as xgb
import catboost as cat
from time import time
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit, RepeatedStratifiedKFold
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import linear_model
# from bayes_opt import BayesianOptimization
# import eli5
# import shap
# from IPython.display import HTML
from category_encoders.ordinal import OrdinalEncoder
import time
from numba import jit
import eli5
import shap
from IPython.display import HTML
import altair as alt
from category_encoders.ordinal import OrdinalEncoder
import networkx as nx

import warnings
warnings.filterwarnings('ignore')

import gc

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 400)

Using TensorFlow backend.
C:\Users\16507\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\16507\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\16507\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\16507\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [2]:
train = pd.read_pickle('train_reduced.pkl')
test = pd.read_pickle('test_reduced.pkl')

print(train.shape)
print(test.shape)

(12497456, 20)
(1156413, 31)


In [22]:
train.head()

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world,month,dayofyear,Phase_Of_Day,time_by_phase_type,practice_sec,months_played,recent_ratio,total_game_time,difficulty
0,27253bdc,45bb1e1b6b50c07b,2019-09-06 17:53:46.937000+00:00,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0.0,Welcome to Lost Lagoon!,Clip,NONE,9,249,Evening,0.00,47804.47,1.0,0.75,1357.0,0.0
1,27253bdc,17eeb7f223665f53,2019-09-06 17:54:17.519000+00:00,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0.0,Magma Peak - Level 1,Clip,MAGMAPEAK,9,249,Evening,0.00,47804.47,1.0,0.75,1357.0,0.0
2,77261ab5,0848ef14a8dc6892,2019-09-06 17:54:56.302000+00:00,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0001e90f,1,2000,0.0,Sandcastle Builder (Activity),Activity,MAGMAPEAK,9,249,Evening,203527.66,47804.47,1.0,0.75,1357.0,1.0
3,b2dba42b,0848ef14a8dc6892,2019-09-06 17:54:56.387000+00:00,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,2,3010,53.0,Sandcastle Builder (Activity),Activity,MAGMAPEAK,9,249,Evening,203527.66,47804.47,1.0,0.75,1357.0,1.0
4,1bb5fbdb,0848ef14a8dc6892,2019-09-06 17:55:03.253000+00:00,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,3,3110,6972.0,Sandcastle Builder (Activity),Activity,MAGMAPEAK,9,249,Evening,203527.66,47804.47,1.0,0.75,1357.0,1.0


In [3]:
grouped = train.groupby('installation_id',as_index=False).last()
grouped.head()

,installation_id,event_id,game_session,timestamp,event_data,event_count,event_code,game_time,title,type,world,month,dayofyear,Phase_Of_Day,time_by_phase_type,practice_sec,months_played,recent_ratio,total_game_time,difficulty
0,0001e90f,27253bdc,04168b61c0650977,2019-09-06 18:18:42.392000+00:00,"{""event_code"": 2000, ""event_count"": 1}",1,2000,0.0,Ordering Spheres,Clip,TREETOPCITY,9,249,Evening,0.00,47804.470,1.0,0.7500,1357.0,0.0
1,000447c4,4d911100,356348f67a1dfb26,2019-10-11 15:03:37.812000+00:00,"{""description"":""Great job! Those thirsty dinos...",124,3121,175347.0,Dino Drink,Game,MAGMAPEAK,10,284,Evening,254881.44,19717.938,1.0,0.9000,181.0,2.0
2,0006a69f,76babcde,0721802df0531701,2019-08-29 16:39:12.615000+00:00,"{""coordinates"":{""x"":1013,""y"":195,""stage_width""...",15,4070,167337.0,Dino Dive,Game,MAGMAPEAK,8,241,Evening,254881.44,25980.172,1.0,0.5000,3800.0,2.0
3,0006c192,27253bdc,df16e87c1003851d,2019-10-01 01:04:36.492000+00:00,"{""event_code"": 2000, ""event_count"": 1}",1,2000,0.0,Crystal Caves - Level 2,Clip,CRYSTALCAVES,10,274,Night,0.00,37441.812,2.0,0.2979,2224.0,0.0
4,0009a5a9,d3640339,d78d2fa8766987f0,2019-09-11 19:17:40.502000+00:00,"{""target_water_level"":3,""water_level"":0,""round...",185,4090,1684228.0,Dino Dive,Game,MAGMAPEAK,9,254,Night,199051.25,129544.620,1.0,0.7500,412.0,2.0


In [ ]:
###for each installation_id, select one Assessment and all game_sessions before then

##with random choice, pick either the min or max(doesn't matter)

##afterwards, focus on how to get all game sessions before

In [32]:
'''truncating the train set to match the test set my randomly picking one assessment and all game sessions
before that assessment for each unique installation'''

##perhaps you dont even need to include (grouped['installation_id']==x)

for x in grouped.installation_id.unique():
    grouped.loc[(grouped['installation_id']==x) & (grouped['type']=='Assessment'),
                 'random'] = np.random.choice(10, len(grouped.loc[(grouped['installation_id']==x)
                                                                   & (grouped['type']=='Assessment')]), replace=False)

In [33]:
grouped.head()

,installation_id,event_id,game_session,timestamp,event_data,event_count,event_code,game_time,title,type,world,month,dayofyear,Phase_Of_Day,time_by_phase_type,practice_sec,months_played,recent_ratio,total_game_time,difficulty,random
0,0001e90f,27253bdc,04168b61c0650977,2019-09-06 18:18:42.392000+00:00,"{""event_code"": 2000, ""event_count"": 1}",1,2000,0.0,Ordering Spheres,Clip,TREETOPCITY,9,249,Evening,0.00,47804.470,1.0,0.7500,1357.0,0.0,NaN
1,000447c4,4d911100,356348f67a1dfb26,2019-10-11 15:03:37.812000+00:00,"{""description"":""Great job! Those thirsty dinos...",124,3121,175347.0,Dino Drink,Game,MAGMAPEAK,10,284,Evening,254881.44,19717.938,1.0,0.9000,181.0,2.0,NaN
2,0006a69f,76babcde,0721802df0531701,2019-08-29 16:39:12.615000+00:00,"{""coordinates"":{""x"":1013,""y"":195,""stage_width""...",15,4070,167337.0,Dino Dive,Game,MAGMAPEAK,8,241,Evening,254881.44,25980.172,1.0,0.5000,3800.0,2.0,NaN
3,0006c192,27253bdc,df16e87c1003851d,2019-10-01 01:04:36.492000+00:00,"{""event_code"": 2000, ""event_count"": 1}",1,2000,0.0,Crystal Caves - Level 2,Clip,CRYSTALCAVES,10,274,Night,0.00,37441.812,2.0,0.2979,2224.0,0.0,NaN
4,0009a5a9,d3640339,d78d2fa8766987f0,2019-09-11 19:17:40.502000+00:00,"{""target_water_level"":3,""water_level"":0,""round...",185,4090,1684228.0,Dino Dive,Game,MAGMAPEAK,9,254,Night,199051.25,129544.620,1.0,0.7500,412.0,2.0,NaN


In [6]:
## maybe get the round total by grabbing group from regular expression
grouped['round']=0

k = 'round'
for e in range(len(grouped)):
    cnt = 0
    x = grouped['event_data'].iloc[e]
    y = json.loads(x)
    for key, value in y.items():
        if key==k:
            cnt+=value
#             grouped['round'].iloc[e]=int(value)
        else:
            cnt=cnt
#             grouped['round'].iloc[e]=0
    grouped['round'].iloc[e]=cnt
#             for key, val in value.items():
# #                 if k in key:

In [ ]:
for i in train['installation_id'].unique():
    if i in grouped['installation_id'].values:
        train['round']= grouped.loc[grouped['installation_id']==i]['round']

In [29]:
train['round'].value_counts()

0    12497456
Name: round, dtype: int64

In [ ]:
train['round']= np.where(train['installation_id'].isin(grouped['installation_id']), grouped['round'], 0)